In [0]:
BASE_PATH_CRM = '/Volumes/dlh/landing_db/raw_sales/source_crm/'
BASE_PATH_ERP = '/Volumes/dlh/landing_db/raw_sales/source_erp/'

INGESTION_CONFIG = [
  {
    "source_table":"crm_customer_table",
    "source_path":"{BASE_PATH_CRM}/cust_info.csv"
    "target_table":"crm_cust_info"
      
  },

  {
    "source_table":"crm_product_info",
    "source_path":"{BASE_PATH_CRM}/prod_info.csv",
    "target_table":"crm_prod_info"
  },
   {
    "source_table":"crm_sales_details",
    "source_path":"{BASE_PATH_CRM}/sales_details.csv",
    "target_table":"crm_sales_details"
  },
  {
    "source_table":"erp",
    "source_path":"{BASE_PATH_ERP}/CUST_AZ12.csv",
    "target_table":"erp_cust_az12"
  },
  {
    "source_table":"erp",
    "source_path":"{BASE_PATH_ERP}/LOC_A101.csv",
    "target_table":"erp_loc_a101"
  },
  {
    "source_table":"erp",
    "source_path":"{BASE_PATH_ERP}/PX_CAT_G1V2.csv",
    "target_table":"erp_px_cat_g1v2"
  }
]


for item in INGESTION_CONFIG:
  print(f"Ingesting {item['source_table']} into {item['target_table']}")
  
  df = (
  spark.read
    .format("csv")
    .option("header","true")
    .option("inferschema","true")
    .load(item['source_path'])
  )
 ( df.write
    .mode('overwrite')
    .format('delta')
    .saveAsTable(f'cdc.sql.{item['target_table']}')
  )
  
  
  
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


base_path = "/Volumes/dlh/landing_db/raw_sales/"

df_all_files = (
    spark.read
      .option("recursiveFileLookup","true")
      .csv(base_path)
      .select(F.col("_metadata.file_path").alias("file_path"))
      .distinct()
)